In [1]:
%region eu-west-1
%profile superuser-test
%iam_role arn:aws:iam::866525145675:role/LSM-DP-Genius-Rec-Glue-Service-Role-test
%glue_version 4.0
%number_of_workers 2
%worker_type G.1X
%idle_timeout 60
%session_id_prefix lsm-dp
%session_type ETL
%extra_jars s3://lsm-dp-utility-eu-test/jre/sqljdbc41.jar
%connections GeniusAnalyticalRep-RDS-test, ODS-Genius-RDS-test

UsageError: Line magic function `%region` not found.


In [ ]:
%%tags 
{
    "lm_app":"aws-dp", 
    "lm_app_env":"test", 
    "lm_sbu":"GS", 
    "lm_troux_uid":"B3439EB1-509F-4193-B1B3-2023D75CBD29", 
    "Name":"lsm-dp-int-glue-sesion-test", 
    "deployment_guid":"2f72a84f-6093-4813-8781-77ca784427b8",
    "organization_guid":"2326c977-2f8d-4d97-8f13-a7ca2fd5d5db"
}

In [ ]:
%%configure 
{
    "--extra-py-files": "s3://lsm-dp-utility-eu-test/python-library/lsm-dp-library.zip"
}

In [ ]:
from awsglue.context import GlueContext
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from extract_utils import *
from py4j.java_gateway import java_import
from pyspark.sql.types import *
from extract_utils import *

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session


In [ ]:
#jdbc:sqlserver://grs-datatech-bi-rds-mssql-bi-lz-uat.cxdi9qpyxrqv.us-east-1.rds.amazonaws.com:1433;databaseName=BI_LZ;
source_connection = "GeniusAnalyticalRep-RDS-qa"
source_db = "BI_LZ"
source_jdbc_conf = glueContext.extract_jdbc_conf(source_connection)
url = "{};databaseName={};".format(source_jdbc_conf.get("url"), source_db)
username = source_jdbc_conf.get("user")
password = source_jdbc_conf.get("password")

java_import(sc._gateway.jvm, "java.sql.Connection")
java_import(sc._gateway.jvm, "java.sql.DatabaseMetaData")
java_import(sc._gateway.jvm, "java.sql.DriverManager")
java_import(sc._gateway.jvm, "java.sql.SQLException")

conn = sc._gateway.jvm.DriverManager.getConnection(url, username, password)
print(conn.getMetaData().getDatabaseProductName())
print(url)
print(username)
print(password)

In [ ]:
java_import(sc._gateway.jvm, "java.sql.Types")
stmnt = "{CALL IDS.GetCurrentCDCUpperBoundary_ID(?)}"
cstmnt = conn.prepareCall(stmnt)
cstmnt.registerOutParameter(1, sc._gateway.jvm.java.sql.Types.VARCHAR)
cstmnt.execute()
rs = cstmnt.getString(1)
cstmnt.close()
conn.close()
print(rs)